# Importing All Libraries

In [1]:
import speech_recognition as sr
import pyttsx3
import soundfile as sf
import openai
import os
import sys

# Initializing All Variables

In [2]:
# Reading API Key
with open("./apikey.txt") as file:
    openai.api_key = file.read()

# Define path to audio output file
output_audio_file = "./output.mp3"

# Initializing text-to-speech object
engine = pyttsx3.init()

# Initialize boolean values
trigger_ernest = True
trigger_response = False

# Define the persona of Ernest
messages = [
    {"role": "system", "content": "You are Ernest, a friendly and empathetic virtual avatar from Standard Chartered Bank that can help recommend one of the bank savings account or current account best suited to customers' needs through meaningful conversations. When customers are lost, he can also explain how to fill up the bank account application form."},
    {"role": "user", "content": "Hi Ernest"},
    {"role": "assistant", "content": "How can I help you?"}
]

# Define functions 

In [3]:
def speak(text):
    engine.say(text)
    engine.runAndWait()
    
def generate_response(transcript):
    
    transcript += '\n\n===\n\n'
    # Append to the conversation
    messages.append({"role": "user", "content": transcript})
    conversation_context = ' '.join([f"{msg['role']}: {msg['content']}" for msg in messages])
    
    # Generate an instance of fine-tuned chatGPT
    response = openai.Completion.create(engine='curie:ft-personal-2023-06-04-10-13-42',
                                        prompt=conversation_context,
                                        temperature=0.7,
                                        max_tokens=50,
                                        n=1,
                                        stop=["\n"],
                                        timeout=None)
    return response.choices[0].text.strip()

# Audio-Response Generation

In [4]:
"""
This part of the code is triggered when the user clicks on the button "speak to Ernest"
"""

def trigger_response():
    try:
        with sr.Microphone() as mic:
            recognizer = sr.Recognizer()
            recognizer.adjust_for_ambient_noise(mic, duration = 1)

            # Line 13 will be replaced by D-ID audio once it is ready to make the voice quality more consistent
            speak("How can I help you?")
            audio = recognizer.listen(mic, phrase_time_limit=None, timeout=None)
            with open(output_audio_file, "wb") as f:
                f.write(audio.get_wav_data(convert_rate=44100, convert_width=2))
            audio_file = open(output_audio_file, "rb")
            transcript = openai.Audio.transcribe("whisper-1", audio_file)["text"]
            print("The transcript is:", transcript)
            response = generate_response(transcript)
            print("The response is:", response)

            # Line 24 will be replaced by D-ID once it is ready
            speak(response)
    except sr.UnknownValueError:
        sys.exit(1)

# Call for Ernest (TBD)

In [5]:
while True:
    try:
        with sr.Microphone() as mic:
            recognizer = sr.Recognizer()
            recognizer.adjust_for_ambient_noise(mic, duration = 0.3)
            audio = recognizer.listen(mic, phrase_time_limit=3)
            call = recognizer.recognize_google(audio).lower()
            print(call)

            if 'ernest' in call or 'honest' in call or 'hyannis' in call or 'earnest' in call:
                trigger_response()
    except sr.UnknownValueError:
        print("No audio detected. Continue detecting in background...")
        continue

No audio detected. Continue detecting in background...
my honest
The transcript is: I need help opening a bank account but I'm not sure which one I should open. Could you suggest some products for me?
The response is: No problem, which type of account are you looking for?
No audio detected. Continue detecting in background...
hey dennis
hey ernest
The transcript is: I'm looking for a current account.
The response is: No problem, which Current Account would you like to apply for?
pianist
heatonist
gabe earnest
The transcript is: What current accounts do you have to recommend to me?
The response is: 
No audio detected. Continue detecting in background...


KeyboardInterrupt: 